In [1]:
# === SETUP ===
import pulp
import os

# Portable solver setup, to allow work locally as well as in JupyterHub
from pulp import COIN_CMD
if os.path.exists('/opt/homebrew/opt/cbc/bin/cbc'):
    solver = COIN_CMD(path='/opt/homebrew/opt/cbc/bin/cbc', msg=0)
else:
    solver = pulp.PULP_CBC_CMD(msg=0)

In [2]:
# Input available information

# Define the greenhouse space available
S = 10000  # square meters

In [3]:
# Space required per container for each plant type
space = {
    'A': 50,  # sq meters
    'B': 80,
    'C': 14,
    'D': 33
}

print(space)

{'A': 50, 'B': 80, 'C': 14, 'D': 33}


In [4]:
# Profit per container
profit = {'A': 20, 'B': 100, 'C': 40, 'D': 80}  # dollars
print(profit)

{'A': 20, 'B': 100, 'C': 40, 'D': 80}


In [5]:
# Time to harvest per container
time = {'A': 20, 'B': 40, 'C': 10, 'D': 32}  # days
print(time)

{'A': 20, 'B': 40, 'C': 10, 'D': 32}


In [6]:
# Initialize the optimization model
# LpMaximize: we want to maximize profit
model = pulp.LpProblem('Greenhouse', pulp.LpMaximize)

In [7]:
# Define decision variables: number of containers to plant for each type
# lowBound=0: can't plant negative containers
# cat='Integer': must plant whole containers
A = pulp.LpVariable('A', lowBound=0, cat='Integer')
B = pulp.LpVariable('B', lowBound=0, cat='Integer')
C = pulp.LpVariable('C', lowBound=0, cat='Integer')
D = pulp.LpVariable('D', lowBound=0, cat='Integer')

In [8]:
# Objective: maximize profit per day (profit/time for each plant type)
# profit['A']/time['A'] = profit rate in $/day for plant A
model += (profit['A']/time['A'])*A + \
         (profit['B']/time['B'])*B + \
         (profit['C']/time['C'])*C + \
         (profit['D']/time['D'])*D

In [9]:
# Constraint: total space used cannot exceed available space
const = space['A']*A + space['B']*B + space['C']*C + space['D']*D
model += const <= S, 'Space'

In [10]:
# Constraint: plant A containers/day >= 2× plant B containers/day
# Convert to per-day basis: A/time['A'] >= 2 * B/time['B']
model += (1.0/time['A']) * A >= (2.0/time['B']) * B, 'AB'

In [11]:
# Constraint: plant B must be at least 20% of total production (on per-day basis)
# B/time['B'] >= 0.2 * (A/time['A'] + B/time['B'] + C/time['C'] + D/time['D'])
model += (1.0/time['B']) * B >= 0.2 * ((1.0/time['A']) * A + (1.0/time['B']) * B + (1.0/time['C']) * C + (1.0/time['D']) * D), 'B'

In [12]:
# Run solver
model.solve(solver)
print(f'Status: {pulp.LpStatus[model.status]}\n')

# Display results
print(model)
print('Maximum profit per day: $', pulp.value(model.objective))
print('Optimal solution:')
for v in model.variables():
    print(f'  {v.name} = {v.varValue:.0f} containers')

Status: Optimal

Greenhouse:
MAXIMIZE
1.0*A + 2.5*B + 4.0*C + 2.5*D + 0.0
SUBJECT TO
Space: 50 A + 80 B + 14 C + 33 D <= 10000

AB: 0.05 A - 0.05 B >= 0

B: - 0.01 A + 0.02 B - 0.02 C - 0.00625 D >= 0

VARIABLES
0 <= A Integer
0 <= B Integer
0 <= C Integer
0 <= D Integer

Maximum profit per day: $ 408.0
Optimal solution:
  A = 56 containers
  B = 56 containers
  C = 3 containers
  D = 80 containers
